In [87]:
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix


In [2]:
train_data = pd.read_csv('sonyorum.csv')

train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47944 entries, 0 to 47943
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   yorum   47944 non-null  object 
 1   puan    47944 non-null  float64
dtypes: float64(1), object(1)
memory usage: 749.2+ KB


In [3]:
import re
## texti temizliyoruz
print("Text before filtering:\nouch...junior is angryð#got7 #junior #yugyo..., @user")
print("\n\nText after filtering:")
re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])", "","ouch...junior is angryð#got7 #junior #yugyo..., @user")

Text before filtering:
ouch...junior is angryð#got7 #junior #yugyo..., @user


Text after filtering:


'ouchjunior is angrygot7 junior yugyo '

In [4]:
train_data.head(10)


,yorum,puan
0,Beğendim 10 numara 8-9 bilgisayara kadar gider...,1.0
1,Urun kalitesi on numara bes yildiz,1.0
2,ürün güzel ama şarj kablosu kalitesiz,1.0
3,Ürünü kartona sarıp göndermek yerine streçe sa...,0.0
4,Dakikalarca uğraşmama rağmen aracım da sadece ...,0.0
5,Ürün harika sadece temizlemesi sorun..,1.0
6,Kılıf fiyatına göre gayet iyi gönül rahatlığıy...,1.0
7,Sadece oyuncular için değil ofiste çalışanlar ...,1.0
8,Güzel ve kaliteli ürün. Kargodaki özende olduk...,1.0
9,1.5 yil once ayni urunden almistim. digerinin ...,1.0


In [5]:
#gereksiz sık kullanılan kelimelerin (stop-words) atılması
def remove_stopwords(df_fon):
    stopwords = open('turkce-stop-words', 'r').read().split()
    df_fon['stopwords_removed'] = list(map(lambda doc:
        [word for word in doc if word not in stopwords], df_fon['yorum']))

remove_stopwords(train_data)

In [6]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_data["yorum"],train_data["puan"], test_size = 0.2, random_state = 42)

In [10]:
print(x_train.head())
print('\n\nX_train shape: ', x_train.shape)

30637    Kia cerato 2006 model aracim için kullandim gü...
39623    Kaliteli  aleminyum ürün kurulumu kolay ellbet...
5082                            guzel ses veriyor ve rahat
41726    Ürün sorunsuzca geldi. Hediye için de teşekkür...
35025                            Kullanıyorum. Gayet güzel
Name: yorum, dtype: object


X_train shape:  (38355,)


In [35]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
count_vect = CountVectorizer(stop_words='english')
transformer = TfidfTransformer(norm='l2',sublinear_tf=True)

In [36]:
x_train_counts = count_vect.fit_transform(x_train)
x_train_tfidf = transformer.fit_transform(x_train_counts)

In [37]:
#EGİTİM SAYILARINI ÖGRENİYORUZ
print(x_train_counts.shape)
print(x_train_tfidf.shape)

(38355, 41200)
(38355, 41200)


In [38]:
x_test_counts = count_vect.transform(x_test)
x_test_tfidf = transformer.transform(x_test_counts)

In [39]:
#TEST SAYILARINI ÖĞRENİYORUZ
print(x_test_counts.shape)
print(x_test_tfidf.shape)

(9589, 41200)
(9589, 41200)


In [40]:
from sklearn.linear_model import LogisticRegression
model1=LogisticRegression()
model1.fit(x_train_tfidf,y_train)

LogisticRegression()

In [41]:
predLR=model1.predict(x_test_tfidf)

In [42]:
from sklearn.metrics import confusion_matrix,f1_score
c_m = confusion_matrix(y_test,predLR)
print(c_m)

[[ 493  481]
 [  26 8589]]


In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predLR)

0.9471269162582125

In [46]:
tp,fn,fp,tn = c_m[0][0],c_m[0][1],c_m[1][0],c_m[1][1]
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1score = 2 * (recall * precision) / (recall + precision)
print("recall = ",recall)
print("precision = ", precision)
print("F1_score = ", f1score)

recall =  0.5061601642710473
precision =  0.9499036608863198
F1_score =  0.6604152712659075


In [93]:
print(classification_report(y_test,predLR))

              precision    recall  f1-score   support

         0.0       0.95      0.51      0.66       974
         1.0       0.95      1.00      0.97      8615

    accuracy                           0.95      9589
   macro avg       0.95      0.75      0.82      9589
weighted avg       0.95      0.95      0.94      9589



In [45]:
while(True):
    yorum=input("Yorumunuz Nedir?(Programdan çıkmak için \'F\' yazınız)")
    if(yorum == 'F' or yorum == 'f'):
        break
    else:
        print(model1.predict(count_vect.transform([yorum])))

Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)güzel
[1.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)kötü
[0.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)berbat
[0.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)rezil
[0.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)f


In [47]:
from sklearn.tree import DecisionTreeClassifier
model2=DecisionTreeClassifier()
model2.fit(x_train_tfidf,y_train)

DecisionTreeClassifier()

In [49]:

predDT=model2.predict(x_test_tfidf)
c_m1 = confusion_matrix(y_test,predDT)
print(c_m1)

[[ 609  365]
 [ 264 8351]]


In [54]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,predDT)



0.9344040045885911

In [55]:
tp,fn,fp,tn = c_m1[0][0],c_m1[0][1],c_m1[1][0],c_m1[1][1]
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1score = 2 * (recall * precision) / (recall + precision)
print("recall = ",recall)
print("precision = ", precision)
print("F1_score = ", f1score)

recall =  0.6252566735112937
precision =  0.697594501718213
F1_score =  0.6594477531131564


In [92]:
print(classification_report(y_test,predDT))

              precision    recall  f1-score   support

         0.0       0.70      0.63      0.66       974
         1.0       0.96      0.97      0.96      8615

    accuracy                           0.93      9589
   macro avg       0.83      0.80      0.81      9589
weighted avg       0.93      0.93      0.93      9589



In [56]:
while(True):
    yorum=input("Yorumunuz Nedir?(Programdan çıkmak için \'F\' yazınız)")
    if(yorum == 'F' or yorum == 'f'):
        break
    else:
        print(model2.predict(count_vect.transform([yorum])))

Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)güzel
[1.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)kötü
[0.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)berbat
[0.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)aşırı iyi
[1.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)f


In [57]:
from sklearn.svm import SVC
model3=SVC()
model3.fit(x_train_tfidf,y_train)

SVC()

In [58]:
predSVC=model3.predict(x_test_tfidf)

In [59]:

c_m2 = confusion_matrix(y_test,predSVC)
print(c_m2)

[[ 546  428]
 [  23 8592]]


In [60]:
accuracy_score(y_test,predSVC)

0.9529669412868912

In [61]:
tp,fn,fp,tn = c_m2[0][0],c_m2[0][1],c_m2[1][0],c_m2[1][1]
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1score = 2 * (recall * precision) / (recall + precision)
print("recall = ",recall)
print("precision = ", precision)
print("F1_score = ", f1score)

recall =  0.5605749486652978
precision =  0.9595782073813708
F1_score =  0.7077122488658457


In [91]:
print(classification_report(y_test,predSVC))

              precision    recall  f1-score   support

         0.0       0.96      0.56      0.71       974
         1.0       0.95      1.00      0.97      8615

    accuracy                           0.95      9589
   macro avg       0.96      0.78      0.84      9589
weighted avg       0.95      0.95      0.95      9589



In [63]:
while(True):
    yorum=input("Yorumunuz Nedir?(Programdan çıkmak için \'F\' yazınız)")
    if(yorum == 'F' or yorum == 'f'):
        break
    else:
        print(model3.predict(count_vect.transform([yorum])))

Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)güzel
[1.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)kötü
[0.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)çirkin
[1.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)en iyi
[1.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)f


In [62]:
from sklearn.ensemble import RandomForestClassifier
model4 = RandomForestClassifier(n_estimators=200)
model4.fit(x_train_tfidf,y_train)

RandomForestClassifier(n_estimators=200)

In [66]:
predRF = model4.predict(x_test_tfidf)

In [67]:
c_m3 = confusion_matrix(y_test,predRF)
print(c_m3)

[[ 525  449]
 [  33 8582]]


In [68]:
accuracy_score(y_test,predRF)

0.9497340702888727

In [69]:
tp,fn,fp,tn = c_m3[0][0],c_m3[0][1],c_m3[1][0],c_m3[1][1]
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1score = 2 * (recall * precision) / (recall + precision)
print("recall = ",recall)
print("precision = ", precision)
print("F1_score = ", f1score)

recall =  0.5390143737166324
precision =  0.9408602150537635
F1_score =  0.6853785900783289


In [90]:
print(classification_report(y_test,predRF))

              precision    recall  f1-score   support

         0.0       0.94      0.54      0.69       974
         1.0       0.95      1.00      0.97      8615

    accuracy                           0.95      9589
   macro avg       0.95      0.77      0.83      9589
weighted avg       0.95      0.95      0.94      9589



In [70]:
while(True):
    yorum=input("Yorumunuz Nedir?(Programdan çıkmak için \'F\' yazınız)")
    if(yorum == 'F' or yorum == 'f'):
        break
    else:
        print(model4.predict(count_vect.transform([yorum])))

Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)iyi
[1.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)kötü
[0.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)güzel
[1.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)çirkin
[1.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)f


In [71]:
from sklearn.neighbors import KNeighborsClassifier
import seaborn as sns

In [72]:
K = [] 
training = [] 
test = [] 
scores = {} 
  
for k in range(2, 10): 
    model5 = KNeighborsClassifier(n_neighbors = k) 
    model5.fit(x_train_tfidf,y_train) 
  
    #training_score = clf.score(x_train_tfidf,y_train) 
    predKNN = model5.predict(x_test_tfidf)
    test_score = model5.score(x_test_tfidf, y_test) 
    K.append(k) 
  
    #training.append(training_score) 
    test.append(test_score)

In [73]:
c_m4 = confusion_matrix(y_test,predKNN)
print(c_m4)

print("\n\nAccuracy:")
print(test)

[[  91  883]
 [   6 8609]]


Accuracy:
[0.9239753884659506, 0.9109396183126499, 0.8819480654917092, 0.9062467410574617, 0.9024924392533111, 0.9076024611534049, 0.9068724580248201, 0.9072896026697257]


In [74]:
tp,fn,fp,tn = c_m4[0][0],c_m4[0][1],c_m4[1][0],c_m4[1][1]
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1score = 2 * (recall * precision) / (recall + precision)
print("recall = ",recall)
print("precision = ", precision)
print("F1_score = ", f1score)

recall =  0.09342915811088295
precision =  0.9381443298969072
F1_score =  0.1699346405228758


In [89]:
print(classification_report(y_test,predKNN))

              precision    recall  f1-score   support

         0.0       0.94      0.09      0.17       974
         1.0       0.91      1.00      0.95      8615

    accuracy                           0.91      9589
   macro avg       0.92      0.55      0.56      9589
weighted avg       0.91      0.91      0.87      9589



In [75]:
while(True):
    yorum=input("Yorumunuz Nedir?(Programdan çıkmak için \'F\' yazınız)")
    if(yorum == 'F' or yorum == 'f'):
        break
    else:
        print(model5.predict(count_vect.transform([yorum])))

Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)iyi
[1.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)kötü
[0.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)güzel
[1.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)çirkin
[1.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)f


In [78]:

from sklearn.naive_bayes import MultinomialNB
model6 = MultinomialNB()
model6.fit(x_train_tfidf,y_train)

MultinomialNB()

In [79]:
predMNB = model6.predict(x_test_tfidf)

In [80]:
c_m5 = confusion_matrix(y_test,predMNB)
print(c_m5)

[[  96  878]
 [   0 8615]]


In [86]:
accuracy_score(y_test,predMNB)

0.9084367504432161

In [81]:
tp,fn,fp,tn = c_m5[0][0],c_m5[0][1],c_m5[1][0],c_m5[1][1]
precision = tp/(tp+fp)
recall = tp/(tp+fn)
f1score = 2 * (recall * precision) / (recall + precision)
print("recall = ",recall)
print("precision = ", precision)
print("F1_score = ", f1score)

recall =  0.09856262833675565
precision =  1.0
F1_score =  0.1794392523364486


In [88]:
print(classification_report(y_test,predMNB))

              precision    recall  f1-score   support

         0.0       1.00      0.10      0.18       974
         1.0       0.91      1.00      0.95      8615

    accuracy                           0.91      9589
   macro avg       0.95      0.55      0.57      9589
weighted avg       0.92      0.91      0.87      9589



In [83]:
while(True):
    yorum=input("Yorumunuz Nedir?(Programdan çıkmak için \'F\' yazınız)")
    if(yorum == 'F' or yorum == 'f'):
        break
    else:
        print(model6.predict(count_vect.transform([yorum])))

Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)iyi
[1.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)kötü
[1.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)güzl
[1.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)berbat
[1.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)iğrenç
[1.]
Yorumunuz Nedir?(Programdan çıkmak için 'F' yazınız)f
